In [2]:
import numpy as np
import random 
from IPython.display import clear_output
import gym
import time
from matplotlib import pyplot as plt

from collections import deque
from keras.layers import Dense , Embedding ,Reshape
from keras import Sequential
from keras.optimizers import Adam

2023-12-04 18:12:16.939949: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 18:12:17.163312: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 18:12:17.164787: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 18:12:18.140961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:


class Agent:
    def __init__(self, environment):
        self._environment = environment
        self._state_size = environment.observation_space.n
        self._action_size = environment.action_space.n
        self._optimizer = Adam(learning_rate=0.01)

        self.replay_memory = deque(maxlen=2000)

        self.exploration_rate = 1
        self.exploration_decay_rete = 0.001
        self.min_exploration_rate = 0.1
        self.discount_rate= 0.99

        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        self.align_networks()

    def _build_compile_model(self):
        model = Sequential()
        model.add(Embedding(self._state_size, 10, input_length=1))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))

        model.compile(loss='mean_squared_error', optimizer=self._optimizer)
        return model

    def align_networks(self):
        self.target_network.set_weights(self.q_network.get_weights())



    def act(self , state):
        random_number = random.uniform(0,1)
        if random_number < self.exploration_rate :
            return self._environment.action_space.sample()
        
        q_values = self.q_network.predict(state)
        return np.argmax(q_values[0])
    
    def store(self, state, action, reward, next_state, terminated):
        self.replay_memory.append((state, action, reward, next_state, terminated))
    
    def train(self , batch_size):
        batch = random.sample(self.replay_memory, batch_size)
        
        for state, action, reward, next_state, terminated in batch:
            
            target = self.q_network.predict(state)
            
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state)
                target[0][action] = reward + self.discount_rate * np.amax(t)
            
            self.q_network.fit(state, target, epochs=1, verbose=0)


In [14]:
env = gym.make("Taxi-v3")

episodes = 10000
epochs_per_episode = 1000

discount_rate = 0.99

# exploration configurations
exploration_rate = 1
exploration_decay_rate = 0.001
min_exploration_rate = 0.1

batch_size = 32
agent = Agent(env)
agent.q_network.summary()

rewards_per_episodes = []

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1, 10)             5000      
                                                                 
 reshape_5 (Reshape)         (None, 10)                0         
                                                                 
 dense_15 (Dense)            (None, 50)                550       
                                                                 
 dense_16 (Dense)            (None, 50)                2550      
                                                                 
 dense_17 (Dense)            (None, 6)                 306       
                                                                 
Total params: 8406 (32.84 KB)
Trainable params: 8406 (32.84 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
for episode in range(episodes):
    state  , _ = env.reset()
    state = np.reshape(state, [1, 1])
    episode_reward = 0 
    done = False

    for epoch in range(epochs_per_episode):
        action = agent.act(state)


        next_state, reward, done, info , _ = env.step(action) 
        next_state = np.reshape(next_state, [1, 1])
        agent.store(state, action, reward, next_state, done)
        
        state = next_state

        episode_reward += reward

        if done :
            agent.align_networks()
            break

        if len(agent.replay_memory) > batch_size:
            agent.train(batch_size)
        
    agent.exploration_rate = agent.min_exploration_rate + \
        (1 - agent.min_exploration_rate) * \
        np.exp(-1*agent.exploration_decay_rate*episode)

    rewards_per_episodes.append(episode_reward)

env.close()


1/1 [==============================] - 0s 94ms/step


/home/sobi/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 [==============================] - 0s 23ms/step


KeyboardInterrupt: 

: 

In [ ]:
plt.plot(rewards_per_episodes)
plt.show()